In [1]:
import requests
zipurl = 'https://mtgjson.com/api/v5/AllPrintings.json.zip'
resp = requests.get(zipurl, verify=False) # verify=False on work computer bc IT proxy / vpn that's in place DO NOT recommend in general

zname = "data/tmp/AllPrintings.json.zip"
zfile = open(zname, 'wb')
zfile.write(resp.content)
zfile.close()

from zipfile import ZipFile

# Create a ZipFile Object and load sample.zip in it
with ZipFile('data/tmp/AllPrintings.json.zip', 'r') as zipObj:
   # Extract all the contents of zip file in different directory
   zipObj.extractall('data/raw')

/Users/nathan-lovin/.local/share/virtualenvs/mtgjson-testing-aGmfpptf/lib/python3.9/site-packages/urllib3/connectionpool.py:1013: InsecureRequestWarning: Unverified HTTPS request is being made to host 'mtgjson.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/1.26.x/advanced-usage.html#ssl-warnings
  warnings.warn(


In [2]:
import pandas as pd
import json

with open('data/raw/AllPrintings.json', encoding="utf8") as f:
    data = json.load(f)

editions = data['data'].keys()
mtgdb_data_date = data['meta']['date']
mtgdb_data_version = data['meta']['version']

In [13]:
set_info = []
card_set = []
card_atomic = []

for edition in editions:
    tmp_set_info = {
        'set_name': data['data'][edition].get('name'),
        'set_code': data['data'][edition].get('code'),
        'release_date': data['data'][edition].get('releaseDate'),
        'base_set_size': data['data'][edition].get('baseSetSize'),
        'cards_total_n': data['data'][edition].get('totalSetSize'),
        'cards_total_n_calc': len(data['data'][edition]['cards']), #add in check if the abs(calc_n - cards_n) <> 0
        'type': data['data'][edition].get('type'),
        'isOnlineOnly': data['data'][edition].get('isOnlineOnly')
    }

    set_info.append(tmp_set_info)
    #print('Number of cards:', cards_n)

    for i in range(1,cards_n):
        # set basic json structure we're going iterate through
        json_structure_cards = data['data'][edition]['cards'][i]

        # build out our tmp single card dictionary
        tmp_card_set = {
            'set_name': data['data'][edition].get('name'),
            'set_code': json_structure_cards.get('setCode'),
            'card_name': json_structure_cards.get('name'),
            'mtgjsonV4Id': json_structure_cards['identifiers'].get('mtgjsonV4Id'), #old_unique_id
            'uuid': json_structure_cards.get('uuid'), # new_unique_id
            'supertypes': json_structure_cards.get('supertypes'),
            'type': json_structure_cards.get('type'),
            'types': json_structure_cards.get('types'),
            'subtypes': json_structure_cards.get('subtypes'),
            'colors': json_structure_cards.get('colors'),
            'variations': json_structure_cards.get('variations'),
            'hasNonFoil': json_structure_cards.get('hasNonFoil'),
            'hasFoil': json_structure_cards.get('hasFoil'),
            'isReprint': json_structure_cards.get('isReprint')
        }

        # append tmp single card dictionary to the growing all cards dictionary
        card_set.append(tmp_card_set)

    # clean up tmp dicts
    del tmp_set_info
    del tmp_card_set

    # break the loop to just grab one set for now
    break

# dump our card dictionary into a pandas d
set_info_df = pd.DataFrame.from_dict(set_info)
card_set_df = pd.DataFrame.from_dict(card_set)


In [14]:
card_set_df

,set_name,set_code,card_name,mtgjson_id,uuid,supertypes,type,types,subtypes,colors,variations,hasNonFoil,hasFoil,isReprint
0,Tenth Edition,10E,Angel of Mercy,9eb2e54c-a12b-5e88-a9c0-d8c84c52d59c,57aaebc1-850c-503d-9f6e-bb8d00d8bf7c,[],Creature — Angel,[Creature],[Angel],[W],[8fd4e2eb-3eb4-50ea-856b-ef638fa47f8a],True,False,True
1,Tenth Edition,10E,Aven Cloudchaser,c0de6fd8-367a-50fb-b2f4-2b8fa5aeb7d9,8ac972b5-9f6e-5cc8-91c3-b9a40a98232e,[],Creature — Bird Soldier,[Creature],"[Bird, Soldier]",[W],[6adaf14d-43e3-521a-adf1-960c808e5b1a],True,False,True
2,Tenth Edition,10E,Ballista Squad,bfbb65b1-f1bb-5355-9495-fb094f9b0782,a69b404f-144a-5317-b10e-7d9dce135b24,[],Creature — Human Rebel,[Creature],"[Human, Rebel]",[W],None,True,True,True
3,Tenth Edition,10E,Bandage,a634f6df-fd74-54d9-a56c-3fdd2ad3e9bf,6d268c95-c176-5766-9a46-c14f739aba1c,[],Instant,[Instant],[],[W],None,True,True,True
4,Tenth Edition,10E,Beacon of Immortality,43f8222f-3fd5-5491-9e76-5b1cc9a86e93,56f4935b-f6c5-59b9-88bf-9bcce20247ce,[],Instant,[Instant],[],[W],None,True,True,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
502,Tenth Edition,10E,Fog Elemental,6cbe5d76-5244-5a2a-b9d1-30272b9e7d7a,6c515488-2a63-5b1e-84a4-987b5fa993af,[],Creature — Elemental,[Creature],[Elemental],[U],[5d45d146-5a42-502e-affb-71f3e81cf7e0],False,True,True
503,Tenth Edition,10E,Mahamoti Djinn,9b06de27-31d8-503b-ae35-b291ea9da383,206c4a07-1442-5fee-acd8-7a8a00da57a4,[],Creature — Djinn,[Creature],[Djinn],[U],[404d6f98-eefb-5dea-8e81-aed2b5956437],False,True,True
504,Tenth Edition,10E,March of the Machines,cfb2cdd2-2a70-5ec8-b4d0-bd9d51c2db75,82306a22-03b6-5cbd-ba72-0002755c4217,[],Enchantment,[Enchantment],[],[U],[60961bae-4655-5a6b-843d-f6180c2c310d],False,True,True
505,Tenth Edition,10E,Mind Bend,9ce04474-1ee6-5ac2-812a-dace4c7331ce,b00ad339-0cc1-59d4-b8a4-c827b6512b6c,[],Instant,[Instant],[],[U],[7cf5654d-d547-50ce-838a-75ff6045fd0f],False,True,True


In [12]:
set_info_df

,set_name,set_code,release_date,base_set_size,cards_total_n,cards_total_n_calc,type,isOnlineOnly
0,Tenth Edition,10E,2007-07-13,383,508,508,core,False
